In [ ]:
import pandas as pd
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt 

## Загрузка и знакомство с данными

Для работы вам понадобятся предобработанные данные датасета для «Прогноза популярности статьи на Хабре».

In [ ]:
df = pd.read_csv('howpop_train.csv')

In [ ]:
df.shape

In [ ]:
df.head(3).T

Избавимся сразу от переменных, названия которых заканчиваются на `_lognorm`. Выберем их с помощью `filter()` и удалим drop-ом:

In [ ]:
df.drop(filter(lambda c: c.endswith('_lognorm'), df.columns), 
        axis = 1,       # axis = 1: столбцы 
        inplace = True) # избавляет от необходимости сохранять датасет

In [ ]:
df.describe().T

In [ ]:
df.describe(include = ['object', 'bool']).T # бинарные и категориальные переменные

In [ ]:
# настройка внешнего вида графиков в seaborn
sns.set_style("dark")
sns.set_palette("RdBu")
sns.set_context("notebook", font_scale = 1.5, 
                rc = { "figure.figsize" : (15, 5), "axes.titlesize" : 18 })

Столбец published (время публикации) содержит строки. Чтобы мы могли работать с этими данными как с датой/временем публикации, приведём их к типу datetime:

In [ ]:
print(df.published.dtype)
df['published'] = pd.to_datetime(df.published, yearfirst = True)
print(df.published.dtype)

In [ ]:
df.head(50).T

Создадим несколько столбцов на основе данных о времени публикации:

In [ ]:
df['year'] = [d.year for d in df.published]
df['month'] = [d.month for d in df.published]

df['dayofweek'] = [d.isoweekday() for d in df.published]
df['hour'] = [d.hour for d in df.published]

Теперь Ваша очередь. В каждом пункте предлагается построить картинку и с ее помощью ответить на вопрос. Конечно, можно попытаться ответить на все вопросы только с Pandas, без картинок, но я советую Вам потренироваться строить (красивые) визуализации.

## 1. В каком месяце (и какого года) было больше всего публикаций?

* март 2016
* март 2015
* апрель 2015
* апрель 2016

In [ ]:
filter_1 = df['year']==2016
filter_2 = df['year']==2015
filter_3 = df['month']==4
filter_4 = df['month']==3
df__ = pd.DataFrame({'год' : ['2016.03','2016.04','2015.03','2015.04'],
                   'количество публикаций' : [int(df.where(filter_1 & filter_4).dropna().shape[0]),
                                    int(df.where(filter_1 & filter_3).dropna().shape[0]),
                                    int(df.where(filter_2 & filter_4).dropna().shape[0]),
                                    int(df.where(filter_2 & filter_3).dropna().shape[0])]})
df__.groupby('год').sum().plot(kind='bar', rot=45);

## 2. Проанализируйте публикации в месяце из предыдущего вопроса

Выберите один или несколько вариантов:

* Один или несколько дней сильно выделяются из общей картины
* На хабре всегда больше статей, чем на гиктаймсе
* По субботам на гиктаймс и на хабрахабр публикуют примерно одинаковое число статей

Подсказки: постройте график зависимости числа публикаций от дня; используйте параметр hue; не заморачивайтесь сильно с ответами и не ищите скрытого смысла :)

In [ ]:
filter_mas = ((filter_1 & filter_3)|(filter_1 & filter_4)|(filter_2 & filter_3)|(filter_2 & filter_4))
df_1 = df.where(filter_mas).dropna()
for i in [((filter_1 & filter_3),'2016.04'),((filter_1 & filter_4),'2016.03'),
          ((filter_2 & filter_3),'2015.04'),((filter_2 & filter_4),'2015.03')]:
    df_2 = df_1.where((i[0])).dropna()
    df_3 =pd.DataFrame({'Количество публикаций' : [ df_2.where(df['dayofweek'] == i).dropna().shape[0]  for i in range(1,8)],
                        'Дни недели' : [i for i in range(1,8)]})
    df_3.groupby('Дни недели').sum().plot(kind='bar', rot=0, title = i[1] );

In [ ]:
filter_mas = ((filter_1 & filter_3)|(filter_1 & filter_4)|(filter_2 & filter_3)|(filter_2 & filter_4))
df_1 = df.where(filter_mas).dropna()
lst = []
lst_2 = []
df['domain'].where(df['domain'] == 'geektimes.ru').dropna().shape[0]
for i in [((filter_1 & filter_3),'2016.04'),((filter_1 & filter_4),'2016.03'),
          ((filter_2 & filter_3),'2015.04'),((filter_2 & filter_4),'2015.03')]:
    df_2 = df_1.where((i[0]))
    lst.append(df_2['domain'].where(df_2['domain'] == 'habrahabr.ru').dropna().shape[0])
    df_2 = df_2.fillna(0)
    lst_2.append(df_2['domain'].where(df_2['domain'] == 'geektimes.ru').dropna().shape[0])
df_3 =pd.DataFrame({'Количество публикаций на Habr' : lst,
                        'Количество публикаций на Geektimes' : lst_2,
                        'Месяцы' : ['2016.04','2016.03','2015.04','2015.03']})

df_3.groupby('Месяцы').sum().plot(kind='bar', rot=0, );

In [ ]:
df_5 = df.fillna(0)
df_4 =pd.DataFrame({'Количество публикаций на Habr' : [df.where(df['domain'] == 'habrahabr.ru').dropna().shape[0]],
                    'Количество публикаций на Geektimes' : [df_5.where(df_5['domain'] == 'geektimes.ru').dropna().shape[0]]})

df_4.groupby('Количество публикаций на Geektimes').plot(kind='bar');

In [ ]:
filter_mas = ((filter_1 & filter_3)|(filter_1 & filter_4)|(filter_2 & filter_3)|(filter_2 & filter_4))
df = df.fillna(0)
df_1 = df.where(filter_mas).dropna()
lst = []
lst_2 = []
filter_5 = df_1['dayofweek'] == 6
df['domain'].where(df['domain'] == 'geektimes.ru').dropna().shape[0]
for i in [((filter_1 & filter_3 & filter_5),'2016.04'),((filter_1 & filter_4 & filter_5),'2016.03'),
          ((filter_2 & filter_3 & filter_5),'2015.04'),((filter_2 & filter_4 & filter_5),'2015.03')]:
    df_2 = df_1.where((i[0]))
    lst.append(df_2['domain'].where(df_2['domain'] == 'habrahabr.ru').dropna().shape[0])
    df_2 = df_2.fillna(0)
    lst_2.append(df_2['domain'].where(df_2['domain'] == 'geektimes.ru').dropna().shape[0])
df_3 =pd.DataFrame({'Количество публикаций на Habr' : lst,
                        'Количество публикаций на Geektimes' : lst_2,
                        'Месяцы' : ['2016.04','2016.03','2015.04','2015.03']})

df_3.groupby('Месяцы').sum().plot(kind='bar', rot=0,title="По субботам" );

In [ ]:
df_5 = df.fillna(0)
filter_6 = df['domain'] == 'habrahabr.ru'
filter_8 = df_5['domain'] == 'geektimes.ru'
filter_7 =  df['dayofweek'] == 6.0
df_4 =pd.DataFrame({'Количество публикаций на Habr' : [df.where(filter_6 & filter_7).dropna().shape[0]],
                    'Количество публикаций на Geektimes' : [df_5.where(filter_8 & filter_7).dropna().shape[0]]})

df_4.groupby('Количество публикаций на Geektimes').plot(kind='bar',title="По субботам за все время");

## 3. Когда лучше всего публиковать статью?

* Больше всего просмотров набирают статьи, опубликованные в 12 часов дня
* У опубликованных в 10 утра постов больше всего комментариев
* Больше всего просмотров набирают статьи, опубликованные в 6 часов утра
* Максимальное число комментариев на гиктаймсе набрала статья, опубликованная в 9 часов вечера
* На хабре дневные статьи комментируют чаще, чем вечерние

In [ ]:
filter_mas = ((filter_1 & filter_3)|(filter_1 & filter_4)|(filter_2 & filter_3)|(filter_2 & filter_4))
df_1 = df.where(filter_mas).dropna()
df_1[['views','hour']].groupby('hour').sum().plot(kind='bar',rot=90);

In [ ]:
filter_mas = ((filter_1 & filter_3)|(filter_1 & filter_4)|(filter_2 & filter_3)|(filter_2 & filter_4))
df_1 = df.where(filter_mas).dropna()
df_1[['comments','hour']].groupby('hour').sum().plot(kind='bar',rot=90);

In [ ]:
filter_mas = ((filter_1 & filter_3)|(filter_1 & filter_4)|(filter_2 & filter_3)|(filter_2 & filter_4))
df_1 = df.where(filter_mas).dropna()
df_1[['views','hour']].groupby('hour').sum().plot(kind='bar',rot=90);

In [ ]:
df_5 = df.fillna(0)
filter_8 = df_5['domain'] == 'geektimes.ru'
filter_7 =  df['dayofweek'] == 6.0
filter_9 = df['hour'] == 21.0
df_7 = df_5.where(filter_8 & filter_7 & filter_9).dropna()
df_7 = df_7[['post_id','comments']].sort_values(by='comments',ascending=False)
df_7.set_index('post_id',inplace=True)
df_7.head(5).plot(kind='bar',rot=90);

In [ ]:
filter_10 = df['domain'] == 'habrahabr.ru'
df_9 = df.where(filter_10).dropna()
df_9[['comments','hour']].groupby('hour').sum().plot(kind='bar',rot=90);

## 4. Кого из топ-20 авторов чаще всего минусуют?¶

* @Mordatyj
* @Mithgol
* @alizar
* @ilya42

In [ ]:
df[['author','votes_minus']].groupby('author').sum().sort_values(by='votes_minus',ascending=False).head(20).plot(kind='barh',rot=0, fontsize = 12);

## 5. Сравните субботы и понедельники

Правда ли, что по субботам авторы пишут в основном днём, а по понедельникам — в основном вечером?

In [ ]:
lst_sat = []
lst_mon = []
for k in [i for i in range(0,24)]:
    df = df.fillna(0)
    filter_ = df['hour'] == k
    filter_1 = df['dayofweek'] == 6.0
    filter_2 = df['dayofweek'] == 1.0
    lst_sat.append(df.where((filter_ & filter_1)).dropna().shape[0])
    lst_mon.append(df.where((filter_ & filter_2)).dropna().shape[0])
data_ = {"Кол-во публикаций в субботу" : lst_sat,
         "Кол-во публикаций в понедельник" : lst_mon,
        'Время дня' : [i for i in range(0,24)]}
df_11 = pd.DataFrame(data_)
df_11.groupby('Время дня').sum().plot(kind='bar',rot=90);